# Genetic Programming for Feature Engineering
- TODO: build the GA script

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import ipdb
from collections import OrderedDict
import time
from itertools import chain
import copy
import re

import chart_studio.plotly as ply
import chart_studio.tools as plytool
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as plyoff
import plotly.subplots as plysub

pd.set_option('display.max_columns', None)

ModuleNotFoundError: No module named 'plytly'

## Generically-useful Functions

In [ ]:
''' generic function to perform weighted random selection '''
def RandomWeightedSelect(keys, wats, randSeed=None):
    '''
    Randomly select an item from a list, according to a set of
    specified weights.
    :param keys: array-like of items from which to select
    :param wats: array-like of weights associated with the input
        keys; must be sorted in descending weight
    :param randSeed: optional random seed for np.random; if no
        randomization is desired, pass 0
    :return selection: selected item
    :return randSeed: random seed used
    '''
    
    # ranodmize, perhaps
    if randSeed != 0:
        if randSeed is None:
            randSeed = int(str(time.time()).split('.')[1])
        np.random.seed(randSeed)
    
    # ensure weights sum to 1
    totWats = sum(wats)
    if totWats != 1:
        wats = [v/totWats for v in wats]
    
    # get the cumulative weights
    cumWats = np.cumsum(wats)
    # get the indices of where the random [0,1] is < the cum weight
    rnd = np.random.rand()
    seld = rnd < cumWats
    
    return [k for (k,s) in zip(keys, seld) if s][0], randSeed

In [ ]:
''' binary arithmetic operations that can be called as functions '''
# addition
def ad(a, b):
    return a+b

# subtraction
def sb(a, b):
    return a-b

# multiplication
def ml(a, b):
    return np.nan_to_num(a * b, posinf=np.nan)

# division
def dv(a, b):
    # check for longest dimensional match
    try:
        lna = len(a)
    except TypeError:
        lna = 1
    try:
        lnb = len(b)
    except TypeError:
        lnb = 1
    # compute
    if (lna == lnb) & (lna == 1):
        # both scalars
        if b != 0:
            res = a/b
        else:
            res = np.nan
    else:
        # at least 1 iterable
        if lna < lnb:
            # a is scalar, b is not
            a = [a]*lnb
        elif lnb < lna:
            # b is scalar, a is not
            b = [b]*lna
        res = np.nan_to_num(a / b, posinf=np.nan)
    return res

# power
def pw(a, b):
    return np.nan_to_num(a ** b, posinf=np.nan)

# minimum
def mn(a, b):
    # check for longest dimensional match
    try:
        lna = len(a)
    except TypeError:
        lna = 1
    try:
        lnb = len(b)
    except TypeError:
        lnb = 1
    # compute
    if (lna == lnb) & (lna == 1):
        # both scalars
        res = min(a, b)
    elif lna == lnb:
        # both iterables
        res = np.where(a < b, a, b)
    elif lna < lnb:
        # a is scalar, b is not
        tmp = [a]*lnb
        res = np.where(tmp < b, tmp, b)
    elif lnb < lna:
        # b is scalar, a is not
        tmp = [b]*lna
        res = np.where(a < tmp, a, tmp)
    return res

# maximum
def mx(a, b):
    # check for longest dimensional match
    try:
        lna = len(a)
    except TypeError:
        lna = 1
    try:
        lnb = len(b)
    except TypeError:
        lnb = 1
    # compute
    if (lna == lnb) & (lna == 1):
        # both scalars
        res = max(a, b)
    elif lna == lnb:
        # both iterables
        res = np.where(a > b, a, b)
    elif lna < lnb:
        # a is scalar, b is not
        tmp = [a]*lnb
        res = np.where(tmp > b, tmp, b)
    elif lnb < lna:
        # b is scalar, a is not
        tmp = [b]*lna
        res = np.where(a > tmp, a, tmp)
    return res

## Function Tree Class and Function Definitions

In [ ]:
''' a node encapsulates an operation / fatures / constant; it remembers it's parent, and knows it's children '''
class Node(object):
    def __init__(self, typ, val, par):
        self.type = typ
        self.value = val
        self.parent = par
        # init the string and left & right to nothing
        self.Str = '%s(%s)'%(self.type, self.value)
        self.left = None
        self.right = None
        self.leftStr = '_'
        self.rightStr = '_'
        
    def __str__(self):
        return '(%s) -> [%s, %s]'%(self.Str, self.leftStr, self.rightStr)
    
    def setLeft(self, L):
        self.left = L
        self.leftStr = '%s(%s)'%(self.left.type, self.left.value)
        
    def setRight(self, R):
        self.right = R
        self.rightStr = '%s(%s)'%(self.right.type, self.right.value)

In [ ]:
''' a tree represents the entire function; it knows the root, level-ordered structure, and depth '''
class Tree(object):
    def __init__(self, root, maxDepth):
        self.root = root
        self.depth = maxDepth # init actual depth with just the max depth allowed for now
        self.struct = None
        self.function = None
        # build the structure dict & function string
        self.GenStruct()
        
    def __str__(self):
        # first build the struct dict, if necessary
        if self.struct is None:
            self.GenStruct()
        # now print
        return '\n'.join(['%d: %s'%(key, '|'.join([str(node.value) for node in val])) for (key, val) in self.struct.items()])
    
    def __repr__(self):
        return self.__str__()

    @staticmethod
    def __RecTreeStruct(currNode, tree, currKey):
        '''
        Recursive tree structuring function; only to be called by TreeStruct
        '''
        # save the node
        this = tree[currKey].copy()
        this.append(currNode)
        tree[currKey] = this
        if (currNode.left is None) & (currNode.right is None):
            return tree

        if currNode.left is not None:
            tree = Tree.__RecTreeStruct(currNode.left, tree, currKey+1)
        if currNode.right is not None:
            tree = Tree.__RecTreeStruct(currNode.right, tree, currKey+1)
        return tree

    def GenStruct(self):
        '''
        Return the function tree structure as a dictionary.
        :return tree: level number-keyed ordered dict of the tree
        '''
        # populate the tree view dict; have to init the dict to a large number, because
        # with crossovers, trees could get very large
        self.struct = dict.fromkeys(range(1000), [])
        self.struct = Tree.__RecTreeStruct(self.root, self.struct, 0)
        # prune it now (remove unused rows)
        for key in list(self.struct.keys()):
            if self.struct[key] == []:
                self.struct.pop(key)
        
        # set the depth
        self.depth = max(self.struct.keys())+1
        
        # now generate the function
        self.function = self.GenFunction()
        
        return self.struct

    def GenFunction(self):
        '''
        Returns a string representation of the function tree as
        a function. This can also act as the hash of a tree.
        :return function: the string of the function
        '''
        
        funcStrings = {}

        # special handling of const or feat root nodes
        if self.root.type != 'ops':
            funcStrings[self.root] = str(self.root.value)
        else:
            # start at the top & climb down the tree
            for currLev in range(self.depth-1, 0, -1):
                nodes = self.struct[currLev]
                # parse the nodes at this level and iterate in pairs
                for indx in range(0, len(nodes), 2):
                    # if there's a func string already defined, use them
                    lVal = funcStrings.get(nodes[indx], str(nodes[indx].value))
                    rVal = funcStrings.get(nodes[indx+1], str(nodes[indx+1].value))
                    # build and store the function string
                    funcStrings[nodes[indx].parent] = nodes[indx].parent.value + '(' + lVal + ',' + rVal + ')'
                    
        return funcStrings[self.root]    

In [ ]:
''' functions for building a new tree '''
def BuildTreeRec(currNode, currDepth, maxDepth, nodeMeta):
    '''
    Recursive tree building function; only to be called by BuildTree
    '''

    # exit if too deep or at a leaf
    if (currDepth == maxDepth) or (currNode.type != 'ops'):
        return currNode    
    # hit one short of max depth, so ensure only consts or feats selected
    if currDepth == (maxDepth-1):
        noOpsK = [k for k in nodeMeta.keys() if k != 'ops']
        noOpsW = [nodeMeta[t][2] for t in noOpsK]
        nodeTypeL, _ = RandomWeightedSelect(noOpsK, noOpsW, 0)
        nodeTypeR, _ = RandomWeightedSelect(noOpsK, noOpsW, 0)
    else:
        nodeTypeL, _ = RandomWeightedSelect(nodeMeta.keys(), [v[2] for v in nodeMeta.values()], 0)
        nodeTypeR, _ = RandomWeightedSelect(nodeMeta.keys(), [v[2] for v in nodeMeta.values()], 0)
        
    # randomly generate the left node
    nodeValuL = nodeMeta[nodeTypeL][0][np.random.randint(nodeMeta[nodeTypeL][1])]
    nodeL = BuildTreeRec(Node(nodeTypeL, nodeValuL, currNode),
                                 currDepth+1, maxDepth, nodeMeta)
    currNode.setLeft(nodeL)

    # randomly generate the right node
    nodeValuR = nodeMeta[nodeTypeR][0][np.random.randint(nodeMeta[nodeTypeR][1])]
    nodeR = BuildTreeRec(Node(nodeTypeR, nodeValuR, currNode),
                                 currDepth+1, maxDepth, nodeMeta)
    currNode.setRight(nodeR)
    
    return currNode

def BuildTree(maxDepth, nodeMeta, verbose=False):
    '''
    Using a set of types of nodes, build a functional tree.
    :param maxDepth: integer maximum depth allowed for the tree (including the root)
    :param nodeMeta: dictionary holding the a tuple of a list of the node values
        allowed, the number of node values allowed, and node weight for random
        selection; keys are node types of 'ops, 'feats', and 'consts'
    :param verbose: optional (default = false) flag to print some info
    :return tree: the complete functional tree
    '''
    
    # randomly generate the root node type & value
    nodeType, _ = RandomWeightedSelect(nodeMeta.keys(), [v[2] for v in nodeMeta.values()], 0)
    nodeValu = nodeMeta[nodeType][0][np.random.randint(nodeMeta[nodeType][1])]
    
    # build the tree
    rootNode = BuildTreeRec(Node(nodeType, nodeValu, None), 0, maxDepth-1, nodeMeta)

    return Tree(rootNode, maxDepth)

## Genetic Programming Functions

In [ ]:
def TreeCrossover(this, that, verbose=False):
    '''
    Cross two trees at a node selected at random.
    :param this: first tree to cross
    :param that: second tree to cross
    :param verbose: optional (default=False) flag to print the crossover node
    :return this_new: new crossed-over tree
    :return that_new: new crossed-over tree
    '''
    
    # first create copies of the input trees
    thisC = copy.deepcopy(this)
    thatC = copy.deepcopy(that)
    
    # get the random crossover points
    thisXoverNode = np.random.permutation(list(chain.from_iterable(thisC.struct.values())))[0]
    if verbose:
        print('This crossover point: %s '%thisXoverNode)
    thatXoverNode = np.random.permutation(list(chain.from_iterable(thatC.struct.values())))[0]
    if verbose:
        print('That crossover point: %s '%thatXoverNode)
    
    # reassign the children
    try:
        if thisXoverNode.parent.right == thisXoverNode:
            thisXoverNode.parent.right = thatXoverNode
        elif thisXoverNode.parent.left == thisXoverNode:
            thisXoverNode.parent.left = thatXoverNode
    except AttributeError:
        # this is a root node, so there is no parent, so no left or right
        pass
    try:    
        if thatXoverNode.parent.right == thatXoverNode:
            thatXoverNode.parent.right = thisXoverNode
        elif thatXoverNode.parent.left == thatXoverNode:
            thatXoverNode.parent.left = thisXoverNode
    except AttributeError:
        # that is a root node, so there is no parent, so no left or right
        pass
    
    # reassign the parents
    thisXoverNode.parent, thatXoverNode.parent = thatXoverNode.parent, thisXoverNode.parent
    
    # if either is an orphan from having been crossed with a root, make it a new tree;
    # otherwise, just rebuild the structure
    if thisXoverNode.parent is None:
        thatC = Tree(thisXoverNode, thisC.depth + thatC.depth)
    else:
        thatC.GenStruct()
    if thatXoverNode.parent is None:
        thisC = Tree(thatXoverNode, thisC.depth + thatC.depth)
    else:
        thisC.GenStruct()

    return thisC, thatC

In [ ]:
def TreeMutate(this, maxDepth, nodeMeta, verbose=False):
    '''
    Mutate a tree at a randomly-selected node.
    :param this: tree to mutate
    :param maxDepth: integer maximum depth allowed for the tree (including the root)
    :param nodeMeta: dictionary holding the a tuple of a list of the node values
        allowed, the number of node values allowed, and node weight for random
        selection; keys are node types of 'ops, 'feats', and 'consts'
    :param verbose: optional (default = false) flag to print some info
    :return this_new: the mutated tree
    '''
    # first create a copy of the input tree
    thisC = copy.deepcopy(this)
    
    # get the random mutation point
    thisMutePoint = np.random.permutation(list(chain.from_iterable(thisC.struct.values())))[0]
    if verbose:
        print('This mutation point: %s '%thisMutePoint)
    
    # mutating this node can have big consequences; potentially as big as creating
    # a new tree - so just do that; but it should be shorter than a full tree
    muty = BuildTree(np.random.randint(1, maxDepth), nodeMeta, verbose)
    if verbose:
        print(muty)
    
    # mutate the input tree
    if thisMutePoint.parent is None:
        # mutation point is the root, so just pass the new tree
        thisC = muty
    else:
        muty = muty.root
        # now graft in the new tree
        if thisMutePoint.parent.right == thisMutePoint:
            thisMutePoint.parent.right = muty
        elif thisMutePoint.parent.left == thisMutePoint:
            thisMutePoint.parent.left = muty
        muty.parent = thisMutePoint.parent
        thisC.GenStruct()

    return thisC

In [ ]:
def RunGP(data, params, nodeMeta, objective, verbose=False, randSeed=None):
    '''
    blah
    '''

    # start time and timestamp
    stt = dt.datetime.now()
    sttT = time.perf_counter()
    tstamp = re.sub('[^0-9]', '', stt.isoformat()[:19])
    
    # parse GP parameters
    popul_size = int(params['popul_size'])
    num_generns = int(params['num_generns'])
    nochange_terminate = int(params['nochange_terminate'])
    convgcrit = float(params['convgcrit'])
    elitism = bool(params['elitism'])
    prob_xover = float(params['prob_xover'])
    mate_type = int(params['mate_type'])
    prob_mutate = float(params['prob_mutate'])
    optim_goal = int(params['optim_goal'])
    plotflag = bool(params['plotflag'])
    printfreq = int(params['printfreq'])
    maxDepth = int(params['maxDepth'])
    
    # parse the objective
    objFunc = objective['function']
    objArgs = objective['arguments'] 
    
    # set the random state
    if randSeed is None:
        randSeed = int(str(time.time()).split('.')[1])
    if verbose:
        print('Random Seed = %d'%randSeed)
    np.random.seed(randSeed)
    
    # display parameters
    dispLine = '#'*42
    print('%s\nGP Started on %s\n%s'%(dispLine, stt.isoformat(), dispLine))
    print('Random Seed: %d'%randSeed)
    print('Maximum # Generations: %0.0f\nMininum # of Generations: %0.0f\nConvergence Criteria: %0.8f'%(num_generns,nochange_terminate,convgcrit))
    if popul_size % 2 == 1:
        popul_size += 1
        print('!!Population Size Increased By 1 to be Even!!')
    print('Population Size: %0.0f'%popul_size)
    print('Mutation Rate: %0.2f\nCrossover Rate: %0.2f'%(prob_mutate,prob_xover))
    print('Mating Method: %s'%['SORTED','ROULETTE'][mate_type - 1])
    print('Elitism is: %s'%['OFF','ON'][elitism])
    print(dispLine)
    if optim_goal == 1:
        print('Objective: MAXIMIZE')
    else:
        print('Objective: MINIMIZE')
    print('Objective Function: %s(%s)'(objFunc, objArgs))
    print(dispLine)
    
    # randomly initialize the population of trees
    population = np.array([None]*popul_size)
    for indx in range(popul_size):
        population[indx] = BuildTree(maxDepth, nodeMeta, True)
        if verbose:
            print('%0d\n%s'%(indx, population[indx]))
    
    # now initialize more things
    # save results by generation
    genScores = np.zeros((num_generns, 2), dtype=float)
    genBest = np.array([None]*num_generns)
    # current generation's best
    bestTree = None
    bestScore = optim_goal*-1*np.Inf
    # previous generation's best
    prevBestTree = None
    prevBestScore = bestScore
    # generations with no improvement termination counter
    termCount = 0
    
    # Begin GP Algorithm Whoo Hoo!
    for genCnt in range(num_generns):
        ''' compute or lookup objective function values '''
        popFitness = np.ones(popul_size, dtype=float)*np.Inf
        for popCnt in range(popul_size):
            if genCnt > 0:
                # check if this tree already evaluated, to save time
                prevEval = np.where(population[popCnt].function == allTrees)
                if prevEval.size == 0:
                    # evalaute
                    objArgs['tree'] = population[popCnt].function
                    popFitness[popCnt] = globals()[objFunc](**objArgs)
                else:
                    # look up existing score
                    popFitness[popCnt] = allScores[prevEval]
            else:
                objArgs['tree'] = population[popCnt].function
                popFitness[popCnt] = globals()[objFunc](**objArgs)
                
        # If optim_goal is (+), this will not change the scores, so the true max will be taken.
        # If optim_goal is (-), the signs will all be changed, and since max(X) = min(-X),
        # taking the maximum will really be taking the minimum.
        optInd = np.argmax(optim_goal*popFitness)
        optVal = popFitness[optInd]
        
        # save some stuff before moving along
        genScores[genCnt,:] = (optVal, np.mean(popFitness[np.isfinite(popFitness)]))
        genBest[genCnt,:] = population[optInd,:]
        
        ''' save all unique trees & their scores '''
        if genCnt == 0:
            # first generation, so create these arrays here
            allScores = popFitness
            allTrees = [tree.function for tree in population]
        else:
            # not first generation, so append to the existing arrays
            allScores = np.append(allscores, pop_fitness)
            allTrees = np.vstack((allchroms, [tree.function for tree in population]))
        # now uust get the unique trees
        uniq, ind = np.unique(allTrees, return_index=True)
        allScores = allScores[ind]
        allTrees = uniq
    
        ''' check for early termination '''
        if optim_goal*genScores[genCnt, 0] > optim_goal*bestScore:
            # this is a better score, so save it and reset the counter
            bestScore = genScores[genCnt, 0]
            bestTree = genBest[genCnt, :]
            termCount = 1
        #elif (optim_goal*genScores[genCnt, 0] < optim_goal*bestScore) and (elitism == False):
        #    # if elitism is off, we can still do early termination with this
        #    termcount += 1
        elif abs(optim_goal*genScores[genCnt, 0] - optim_goal*bestScore) < convgcrit:
            # "no" improvement
            termcount += 1
        elif (elitism  == True):
            # with elitism on and a deterministic objective, performance is monotonically non-decreasing
            termcount += 1

        if termCount >= nochange_terminate:
            print('Early Termination On Generation %d of %d'%(genCnt + 1, num_generns))
            genScores = genScores[:(genCnt + 1), :] # keep only up to gencnt spaces (inclusive)
            genBest = genBest[:(genCnt + 1), :]
            break
            
        # don't bother with the next generation
        if genCnt == (num_generns - 1):
            break
    

    
    
    
    # TODO: iterate over generations and:
    # pair off individuals for crossover
    # mutate individuals
    # perform GA engineering (is this really doable in GP?)
    # finish building next generation (elitism?)
    # store and report progress
    
    # stop time
    stp = dt.datetime.now()
    stpT = time.perf_counter()
    print('GP: Started on %s\n\tFinished on %s\n\tElapsed Time = %0.3f(m)'%(stt.isoformat(), stp.isoformat, (stpT-sttT)/60))
    

In [ ]:
def objFunc(data, tree):
    '''
    template objective function
    '''
    
    return 0

## Build Trees

In [ ]:
# set the possible node values
ops = ['ad', 'sb', 'ml', 'dv', 'pw', 'mx', 'mn']
feats = ['X%d'%i for i in range(5)]
consts = [0, 1, 2, 3, 10, 100]

# must be orderd by descending weight - [values, length, weight] 
nodeMeta = OrderedDict()
nodeMeta['ops'] = [ops, len(ops), 0.5]
nodeMeta['feats'] = [feats, len(feats), 0.25]
nodeMeta['consts'] = [consts, len(consts), 0.25]

In [ ]:
''' randomly generate some trees '''
# set the prng seed
randSeed = 8013728#int(str(time.time()).split('.')[1])
print('Random Seed = %d'%randSeed)
np.random.seed(randSeed)

# set the depth
maxDepth = 10

# build the tree, starting from the top node
treeCnt = 20
trees = [None]*treeCnt
for indx in range(treeCnt):
    print('Creating tree %0d'%indx)
    time.sleep(np.random.rand()) # setting a random wait time to allow seed differentiation
    trees[indx] = BuildTree(maxDepth, nodeMeta, True)
    print(trees[indx])

In [ ]:
''' try some GP operations '''
# crossover 2 pairs
trees.extend(TreeCrossover(trees[8], trees[14], True))
trees.extend(TreeCrossover(trees[15], trees[16], True))

# mutate 2
trees.append(TreeMutate(trees[18], maxDepth, nodeMeta, True))
trees.append(TreeMutate(trees[19], maxDepth, nodeMeta, True))

In [ ]:
print(trees[13])
print('--------')
print(trees[16])
print('--------')
print(trees[-4])
print('--------')
print(trees[-3])

## Apply to a Dataframe

In [ ]:
# generate some data
p = len(feats)
n = 1000
data = pd.DataFrame(data=np.random.rand(n, p), columns=feats)
display(data.head())

In [ ]:
# now apply all trees
for indx in range(len(trees)):
    print('Processing tree %0d'%indx)
    func = trees[indx].GenFunction()
    data['tree%0d'%indx] = eval(func.replace('X', 'data.X'))
# talk
display(data.head())